In [99]:
import pandas as pd
import numpy as np
import datetime as dt
from datetime import date

In [100]:
freq = ['daily', 'weekly', 'monthly']
files = [f'../Data/gspc_{f}_clean.csv' for f in freq]
dfs = [pd.read_csv(file) for file in files]

for df in dfs:
    df['date'] = pd.to_datetime(df['date']).dt.normalize()

daily = dfs[0]
weekly = dfs[1]
monthly = dfs[2]

In [101]:
t = pd.read_csv(f'../Data/tweet_df_cleaned_w_sentiment.csv')
t.rename(columns={
        'text': 'tweet', 
        'avg_scores': 'avg', 
        'sum_scores': 'sum', 
        'date': 'timestamp',
        'sentiment_category': 'sentiment',
        'word_count': 'words'
        }, inplace=True)
#tweets.columns
columns = [
        'timestamp', 
        'tweet', 
        'avg', 
        'sum', 
        'isRetweet', 
        'isDeleted', 
        'favorites', 
        'retweets',
        'sentiment',
        'words'
        ]

t['isRetweet'] = t['isRetweet'].apply(lambda x: 1 if x == 't' else 0)
t['isDeleted'] = t['isDeleted'].apply(lambda x: 1 if x == 't' else 0)

#t = t[columns][(t['isRetweet'] == 'f') & (t['isDeleted'] == 'f')].reset_index()
t['date'] = pd.to_datetime(t['timestamp']).dt.normalize()
t['month'] = t['date'] + pd.offsets.MonthBegin(-1)
t['year'] = pd.DatetimeIndex(t['timestamp']).year

sentiments = ['positive', 'negative', 'neutral']
for s in sentiments:
        t[s] = np.where(t['sentiment'] == s, 1, 0)

#t2 = t.groupby('month').agg()

m = t[t['year'] >= 2016]\
        .groupby('month')\
        .agg({
                'tweet': 'count',
                'isRetweet': 'sum',
                'isDeleted': 'sum', 
                'avg': 'mean', 
                'sum': 'sum', 
                'retweets': 'sum', 
                'favorites': 'sum', 
                'words': 'sum',
                'positive': 'sum',
                'negative': 'sum',
                'neutral': 'sum'
        })\
        .reset_index()

d = t[t['year'] >= 2016]\
        .groupby('date')\
        .agg({
                'tweet': 'count',
                'isRetweet': 'sum',
                'isDeleted': 'sum', 
                'avg': 'mean', 
                'sum': 'sum', 
                'retweets': 'sum', 
                'favorites': 'sum', 
                'words': 'sum',
                'positive': 'sum',
                'negative': 'sum',
                'neutral': 'sum'
        })\
        .reset_index()

mo = pd.merge(m, monthly, how='inner', left_on='month', right_on='date')
mo = mo[['month', 'tweet', 'isRetweet', 'isDeleted', 'avg', 'sum', 'retweets', 'favorites', 'words', 'positive', 'negative', 'neutral', '%change', 'up/down']]\
        .reset_index()

da = pd.merge(d, daily, how='inner', on='date')
da = da[['date', 'tweet', 'isRetweet', 'isDeleted', 'avg', 'sum', 'retweets', 'favorites', 'words', 'positive', 'negative', 'neutral', '%change', 'up/down']]\
        .reset_index()

In [102]:
mo.rename(columns={
    'tweet': 'tweets',
    'avg': 'avg_sentiment_score',
    'sum': 'sum_sentiment_score',
    'words': 'word_count',
    '%change': 's&p_%change',
    'up/down': 's&p_up/down'
}, inplace=True)

da.rename(columns={
    'tweet': 'tweets',
    'avg': 'avg_sentiment_score',
    'sum': 'sum_sentiment_score',
    'words': 'word_count',
    '%change': 's&p_%change',
    'up/down': 's&p_up/down'
}, inplace=True)

In [103]:
# mo.to_csv('../Data/monthly_summary.csv', index=False)
# da.to_csv('../Data/daily_summary.csv', index=False)